In [1]:
import requests
import json
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.io as pio

In [2]:
def get_apikey():
    payload = {
        "nickname":"astra",
        "password":"W%Auity!MM(n"
    }

    try:
        response = requests.post('https://api-x90k.onrender.com/v1/auth/login', json=payload)
        response.raise_for_status()
        data = response.json()
        data = data['data']
        return data['api_key']

    except requests.exceptions.HTTPError as err:
        print(err)
    except requests.exceptions.ConnectionError:
        print("Error de conexión")
    except requests.exceptions.Timeout:
        print("Se agotó el tiempo de la solicitud")
    except Exception as e:
        print("Error inesperado: ", str(e))

    return None

# 1. Recolección de datos para categoria 50 y más

In [3]:
api_key = get_apikey()
if api_key is None:
  print("No se pudo obtener la API key.")
  exit()

headers = {
    'X-API-KEY': api_key
}

params = {
    'registration_category_id': 'aacace26-65b6-4ac9-8e7c-fcb32061c3fd'
}

# get the first call for seniors cathegory
try:
    response = requests.get('https://api-x90k.onrender.com/v1/groups/matches/pairs/players', headers=headers, params=params)

    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        # Verificar si la respuesta tiene contenido
        if response.text.strip():
            first_call_seniors = response.json()
        else:
            print("La respuesta está vacía")
    else:
        print("Error:", response.text, response.status_code)

except requests.exceptions.RequestException as e:
    print("Error en la solicitud:", e)

Creo que cada llamada trae los partidos de 4 grupos, entonces para 50 y más solo es necesaria la first call para obtener todos los datos

In [4]:
rows = []

for group in first_call_seniors['data']:
    # Use description because already hace only the group letter
    group_letter = group['group']['description']

    for match_info in group['matches']:
        match = match_info['match']
        pairs = match.get('pairs', [])

        if len(pairs) == 2:
            # team 1
            pair1 = pairs[0]
            players1 = [p['player']['fullname'] for p in pair1['pair']['players']]
            team1_id = pair1['pair']['id']
            points1 = pair1['relationship']['score']

            # team 2
            pair2 = pairs[1]
            players2 = [p['player']['fullname'] for p in pair2['pair']['players']]
            team2_id = pair2['pair']['id']
            points2 = pair2['relationship']['score']

            rows.append({
                'group': group_letter,
                'team1_id': team1_id,
                'team1': f"[{', '.join(players1)}]",
                'points_team1': points1,
                'team2_id': team2_id,
                'team2': f"[{', '.join(players2)}]",
                'points_team2': points2
            })

df_seniors = pd.DataFrame(rows)
df_seniors = df_seniors[['group', 'team1_id' , 'team1', 'points_team1', 'team2_id' , 'team2', 'points_team2']]
df_seniors.columns = ['group', 'team1_id' , 'team1', 'points_team1', 'team2_id' , 'team2', 'points_team2']

In [5]:
df_seniors

,group,team1_id,team1,points_team1,team2_id,team2,points_team2
0,A,8a9e8890-5e67-4300-8792-c18e4fe123f8,"[Alejandro Guzmán, Rodolfo Pérez]",10,ff43b8fd-51e5-47ce-ba80-8d428b61c5b9,"[Juvenal Labra, Oscar Chavez]",6
1,A,0e4ff1f4-19db-4380-a57c-62881b39e146,"[Marco Belmonte, Servando Rincón]",10,ff43b8fd-51e5-47ce-ba80-8d428b61c5b9,"[Juvenal Labra, Oscar Chavez]",7
2,A,ff43b8fd-51e5-47ce-ba80-8d428b61c5b9,"[Juvenal Labra, Oscar Chavez]",0,4e9579ca-4b9e-4018-bb02-5ea047a1938b,"[Alfredo Manriquez, Alberto Chacón]",0
3,A,0e4ff1f4-19db-4380-a57c-62881b39e146,"[Marco Belmonte, Servando Rincón]",0,8a9e8890-5e67-4300-8792-c18e4fe123f8,"[Alejandro Guzmán, Rodolfo Pérez]",10
4,A,4e9579ca-4b9e-4018-bb02-5ea047a1938b,"[Alfredo Manriquez, Alberto Chacón]",7,8a9e8890-5e67-4300-8792-c18e4fe123f8,"[Alejandro Guzmán, Rodolfo Pérez]",8
5,A,0e4ff1f4-19db-4380-a57c-62881b39e146,"[Marco Belmonte, Servando Rincón]",3,4e9579ca-4b9e-4018-bb02-5ea047a1938b,"[Alfredo Manriquez, Alberto Chacón]",10
6,B,4f473639-0ad1-4984-b02a-1861d1c43324,"[Nery Contreras, José Gpe Fierros]",6,af59fb07-0bc4-4148-8fa0-ab9971c396fe,"[Fernando González, Patricio Correa]",7
7,B,4f473639-0ad1-4984-b02a-1861d1c43324,"[Nery Contreras, José Gpe Fierros]",2,378f5c67-cd7c-4ab3-9b8f-50acf38f1661,"[Carlos Labra, Antonio CBaby]",10
8,B,4f473639-0ad1-4984-b02a-1861d1c43324,"[Nery Contreras, José Gpe Fierros]",10,c5e42862-c78a-4802-8a07-4394d5b177df,"[Antonio Melendez, Leopoldo Moya]",2
9,B,af59fb07-0bc4-4148-8fa0-ab9971c396fe,"[Fernando González, Patricio Correa]",0,378f5c67-cd7c-4ab3-9b8f-50acf38f1661,"[Carlos Labra, Antonio CBaby]",10


# Paridad entre grupos de 50 y más

In [6]:
#Make the stats for separate and anfter join them
df_team1 = df_seniors[['group', 'team1_id', 'team1', 'points_team1']]
df_team1 = df_team1.rename(columns={
    'team1_id': 'team_id',
    'team1': 'team',
    'points_team1': 'points_scored'
})
df_team1['points_received'] = df_seniors['points_team2']

df_team2 = df_seniors[['group', 'team2_id', 'team2', 'points_team2']]
df_team2 = df_team2.rename(columns={
    'team2_id': 'team_id',
    'team2': 'team',
    'points_team2': 'points_scored'
})
df_team2['points_received'] = df_seniors['points_team1']

df_paridad_seniors = pd.concat([df_team1, df_team2], ignore_index=True)

In [7]:
df_paridad_seniors = df_paridad_seniors.groupby(['group', 'team_id', 'team']).agg({
    'points_scored': 'sum',
    'points_received': 'sum'
}).reset_index()
df_paridad_seniors = df_paridad_seniors.sort_values(['group', 'points_scored']).reset_index(drop=True)

In [8]:
df_paridad_seniors['points_diff'] = abs(df_paridad_seniors['points_scored'] - df_paridad_seniors['points_received'])

In [9]:
df_paridad_seniors

,group,team_id,team,points_scored,points_received,points_diff
0,A,0e4ff1f4-19db-4380-a57c-62881b39e146,"[Marco Belmonte, Servando Rincón]",13,27,14
1,A,ff43b8fd-51e5-47ce-ba80-8d428b61c5b9,"[Juvenal Labra, Oscar Chavez]",13,20,7
2,A,4e9579ca-4b9e-4018-bb02-5ea047a1938b,"[Alfredo Manriquez, Alberto Chacón]",17,11,6
3,A,8a9e8890-5e67-4300-8792-c18e4fe123f8,"[Alejandro Guzmán, Rodolfo Pérez]",28,13,15
4,B,af59fb07-0bc4-4148-8fa0-ab9971c396fe,"[Fernando González, Patricio Correa]",17,25,8
5,B,4f473639-0ad1-4984-b02a-1861d1c43324,"[Nery Contreras, José Gpe Fierros]",18,19,1
6,B,c5e42862-c78a-4802-8a07-4394d5b177df,"[Antonio Melendez, Leopoldo Moya]",18,30,12
7,B,378f5c67-cd7c-4ab3-9b8f-50acf38f1661,"[Carlos Labra, Antonio CBaby]",30,9,21
8,C,d70aef35-368b-4197-8c9c-cbca64866c10,"[Rafael Aleman, Abraham Aguiñiga]",15,30,15
9,C,6f288107-8b7c-48b0-b7e7-abc81a974be1,"[Antonio Mendoza, Victor Arellano]",19,29,10


In [10]:
df_paridad_seniors.to_csv('/content/drive/MyDrive/TDLF2025/DataCharts/data_matches_seniors.csv', index=False, encoding='utf-8')

In [11]:
# Agrupamos los datos por group_id y calculamos la desviación estándar de la diferencia entre los puntos hechos y recibidos
std_por_grupo = df_paridad_seniors.groupby('group')['points_diff'].std().round(2)

In [12]:
std_por_grupo.rename('parity_index', inplace=True)

,parity_index
group,
A,4.65
B,8.35
C,2.89
D,4.97


In [13]:
std_por_grupo = pd.DataFrame(std_por_grupo). reset_index()

In [14]:
std_por_grupo.to_json('/content/drive/MyDrive/TDLF2025/DataCharts/parity_seniors.json', orient='records')

In [15]:
# Crear la figura
fig = go.Figure()

# Agregar los datos al gráfico de barras
fig.add_trace(go.Bar(x=std_por_grupo['group'],
                     y=std_por_grupo['parity_index'],
                     marker_color='#c22322',  # Color de las barras
                     text=std_por_grupo.values,  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

fig.update_layout(title='¿Qué tan parejos estuvieron los grupos en la categoría 50 y más?',
                  xaxis_title='Grupo',
                  yaxis_title='Indice de paridad',
                  plot_bgcolor='#241111',  # Fondo negro
                  paper_bgcolor='#000000',  # Fondo del área del gráfico negro
                  font=dict(color='white'),  # Color del texto
                  width=1080,
                  height=720,
                  )

# Ajustar el espaciado entre las etiquetas de los ticks del eje x
fig.update_xaxes(dtick=1)  # Establecer dtick en 1 para mostrar todas las etiquetas

# Guardar y mostrar el gráfico
pio.show(fig)

# Funcion general para extraer grupos

In [16]:
def extract_group(df,letter):
  x_group = df[df['group'] == letter].copy()
  x_group.drop(['group', 'team_id', 'points_diff'], axis=1, inplace=True)
  x_group['team'] = x_group['team'].apply(lambda x: '-'.join(x.strip('[]').split(', ')))
  x_group.reset_index(drop=True, inplace=True)
  return x_group

# Extracción de parejas por grupo (50 y más) puntos hechos VS puntos recibidos

**Grupo A**

In [17]:
group_a_seniors = extract_group(df_paridad_seniors, 'A')
group_a_seniors

,team,points_scored,points_received
0,Marco Belmonte-Servando Rincón,13,27
1,Juvenal Labra-Oscar Chavez,13,20
2,Alfredo Manriquez-Alberto Chacón,17,11
3,Alejandro Guzmán-Rodolfo Pérez,28,13


**Grupo B**

In [18]:
group_b_seniors = extract_group(df_paridad_seniors, 'B')
group_b_seniors

,team,points_scored,points_received
0,Fernando González-Patricio Correa,17,25
1,Nery Contreras-José Gpe Fierros,18,19
2,Antonio Melendez-Leopoldo Moya,18,30
3,Carlos Labra-Antonio CBaby,30,9


**Grupo C**

In [19]:
group_c_seniors = extract_group(df_paridad_seniors, 'C')
group_c_seniors

,team,points_scored,points_received
0,Rafael Aleman-Abraham Aguiñiga,15,30
1,Antonio Mendoza-Victor Arellano,19,29
2,Joaquin López-Salvador Farías,25,10
3,Javier Zavala-Jorge Zavala,27,17


**Grupo D**

In [20]:
group_d_seniors = extract_group(df_paridad_seniors, 'D')
group_d_seniors

,team,points_scored,points_received
0,Ricardo Morales-Gustavo Jiménez,3,20
1,Refugio Castillo-Ulises Asencio,13,18
2,Humberto Martínez-Marco Martínez,26,14
3,David Godínez-Alfredo Rocha,27,17


## Puntos hechos VS Puntos recibidos (50 y más)

In [21]:
fig = go.Figure()

# Agregar los datos de los puntos hechos al gráfico de barras
fig.add_trace(go.Bar(x=group_a_seniors['team'],
                     y=group_a_seniors['points_scored'],
                     name = 'Puntos hechos',
                     marker_color='#06FF74',  # Color de las barras
                     text=group_a_seniors['points_scored'],  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

# Agregar los datos de los puntos recibidos al gráfico de barras
fig.add_trace(go.Bar(x=group_a_seniors['team'],
                     y=group_a_seniors['points_received'],
                     name = 'Puntos recibidos',
                     marker_color='#c22322',  # Color de las barras
                     text=group_a_seniors['points_received'],  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

fig.update_layout(title='Puntos totales por equipo',
                  xaxis_title='Equipo',
                  yaxis_title='Puntos',
                  #xaxis_tickangle=-90,
                  plot_bgcolor='#000000',  # Fondo negro
                  paper_bgcolor='#000000',  # Fondo del área del gráfico negro
                  font=dict(color='white'),  # Color del texto
                  width=1280,
                  height=720,
                  )

# Ajustar el espaciado entre las etiquetas de los ticks del eje x
fig.update_xaxes(dtick=1)  # Establecer dtick en 1 para mostrar todas las etiquetas

# Guardar y mostrar el gráfico
pio.show(fig)

# Sinergia por pareja (50 y más)

$(\frac{Ph - Pr + PJ}{Cs})*100$

Donde:

* $Ph$ = Puntos hechos por el equipo
* $Pr$ = Puntos recibidos por el equipo
* $PJ$ = Partidos Jugados por el equipo (para dar ese pequeño empujon)
* $Cs$ = Constante de sinergia que representa una sinergia perfecta (30)

**Se multipplica por 100 ya que la sinergia de un equipo se mide en porcentaje**

In [22]:
sinergia_df_seniors = df_paridad_seniors[['team','points_scored', 'points_received']].copy()
sinergia_df_seniors['sinergy'] = (((sinergia_df_seniors['points_scored'] - sinergia_df_seniors['points_received']+3) / 30) * 100).round(2)
sinergia_df_seniors['team'] = sinergia_df_seniors['team'].apply(lambda x: '-'.join(x.strip('[]').split(', ')))
sinergia_df_seniors

,team,points_scored,points_received,sinergy
0,Marco Belmonte-Servando Rincón,13,27,-36.67
1,Juvenal Labra-Oscar Chavez,13,20,-13.33
2,Alfredo Manriquez-Alberto Chacón,17,11,30.00
3,Alejandro Guzmán-Rodolfo Pérez,28,13,60.00
4,Fernando González-Patricio Correa,17,25,-16.67
5,Nery Contreras-José Gpe Fierros,18,19,6.67
6,Antonio Melendez-Leopoldo Moya,18,30,-30.00
7,Carlos Labra-Antonio CBaby,30,9,80.00
8,Rafael Aleman-Abraham Aguiñiga,15,30,-40.00
9,Antonio Mendoza-Victor Arellano,19,29,-23.33


In [23]:
# Crear una figura de Plotly
fig = go.Figure()

# Agregar barras para la sinergia positiva
fig.add_trace(go.Bar(
    x=sinergia_df_seniors['team'],
    y=sinergia_df_seniors['sinergy'].apply(lambda x: max(x, 0)),  # Tomar solo los valores positivos
    name='Sinergia Positiva',
    marker_color='rgba(0, 111, 231, 0.9)',  # Color para las barras positivas
    text=sinergia_df_seniors['sinergy'].apply(lambda x: f'{x}%'),  # Texto que se mostrará encima de cada barra
    textposition='auto',  # Posición del texto
    textfont=dict(color='white')
))

# Agregar barras para la sinergia negativa
fig.add_trace(go.Bar(
    x=sinergia_df_seniors['team'],
    y=sinergia_df_seniors['sinergy'].apply(lambda x: min(x, 0)),  # Tomar solo los valores negativos
    name='Sinergia Negativa',
    marker_color='rgba(254, 107, 9, 0.9)',  # Color para las barras negativas
    text=sinergia_df_seniors['sinergy'].apply(lambda x: f'{x}%'),  # Texto que se mostrará encima de cada barra
    textposition='auto',  # Posición del texto
    textfont=dict(color='white')
))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Sinergia de Equipos',
    xaxis=dict(title='Equipo'),
    yaxis=dict(title='Sinergia'),
    xaxis_tickangle=-90,
    plot_bgcolor='#000000',  # Fondo negro
    paper_bgcolor='#000000',  # Fondo del área del gráfico negro
    font=dict(color='white'),  # Color del texto
    width=1280,
    height=720
)

fig.update_xaxes(dtick=1)

# Mostrar el gráfico
fig.show()

# ======================SEPARADOR CATEGORIA=====================

# Recolección de datos para categoría Libre

In [24]:
api_key = get_apikey()
if api_key is None:
  print("No se pudo obtener la API key.")
  exit()

headers = {
    'X-API-KEY': api_key
}

params = {
    'registration_category_id': '3722d2f9-bbd3-457c-a8c0-ae47749eadae'
}

# get the first call for seniors cathegory
try:
    response = requests.get('https://api-x90k.onrender.com/v1/groups/matches/pairs/players', headers=headers, params=params)

    # Verificar el código de estado de la respuesta
    if response.status_code == 200:
        # Verificar si la respuesta tiene contenido
        if response.text.strip():
            first_call_open = response.json()
        else:
            print("La respuesta está vacía")
    else:
        print("Error:", response.text, response.status_code)

except requests.exceptions.RequestException as e:
    print("Error en la solicitud:", e)

In [25]:
#Get the matches from all pages
api_key = get_apikey()
if api_key is None:
  print("No se pudo obtener la API key.")
  exit()

headers = {
    'X-API-KEY': api_key
}
pages = first_call_open["pagination"]["total"]
matches_open = []

for i in range(1, pages + 1):
    params = {
        "page": i,
        'registration_category_id': '3722d2f9-bbd3-457c-a8c0-ae47749eadae'
    }
    response = requests.get("https://api-x90k.onrender.com/v1/groups/matches/pairs/players", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        matches_open.extend(data["data"])
    else:
        print("Error:", response.text, response.status_code)
        break

In [26]:
rows_open = []

for group in matches_open:
    # Use description because already hace only the group letter
    group_letter = group['group']['description']

    for match_info in group['matches']:
        match = match_info['match']
        pairs = match.get('pairs', [])

        if len(pairs) == 2:
            # team 1
            pair1 = pairs[0]
            players1 = [p['player']['fullname'] for p in pair1['pair']['players']]
            team1_id = pair1['pair']['id']
            points1 = pair1['relationship']['score']

            # team 2
            pair2 = pairs[1]
            players2 = [p['player']['fullname'] for p in pair2['pair']['players']]
            team2_id = pair2['pair']['id']
            points2 = pair2['relationship']['score']

            rows_open.append({
                'group': group_letter,
                'team1_id': team1_id,
                'team1': f"[{', '.join(players1)}]",
                'points_team1': points1,
                'team2_id': team2_id,
                'team2': f"[{', '.join(players2)}]",
                'points_team2': points2
            })

df_open = pd.DataFrame(rows_open)
df_open = df_open[['group', 'team1_id' , 'team1', 'points_team1', 'team2_id' , 'team2', 'points_team2']]
df_open.columns = ['group', 'team1_id' , 'team1', 'points_team1', 'team2_id' , 'team2', 'points_team2']

In [27]:
df_open

,group,team1_id,team1,points_team1,team2_id,team2,points_team2
0,A,5083a5a9-e528-4995-a0f4-8ef47e26c2f0,"[Gerardo Martínez, Jesús Arguellas]",10,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",1
1,A,78062676-c1f6-4d63-bb8d-3249f2f493d1,"[José Manuel Vargas, Ernesto Oñate]",3,5083a5a9-e528-4995-a0f4-8ef47e26c2f0,"[Gerardo Martínez, Jesús Arguellas]",10
2,A,5242ad66-9a87-4135-bb39-c80570496f4a,"[Jorge Rios, JR PAREJA DOS]",2,5083a5a9-e528-4995-a0f4-8ef47e26c2f0,"[Gerardo Martínez, Jesús Arguellas]",10
3,A,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",3,78062676-c1f6-4d63-bb8d-3249f2f493d1,"[José Manuel Vargas, Ernesto Oñate]",10
4,A,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",10,5242ad66-9a87-4135-bb39-c80570496f4a,"[Jorge Rios, JR PAREJA DOS]",7
5,A,5242ad66-9a87-4135-bb39-c80570496f4a,"[Jorge Rios, JR PAREJA DOS]",5,78062676-c1f6-4d63-bb8d-3249f2f493d1,"[José Manuel Vargas, Ernesto Oñate]",10
6,B,ff951871-2e1e-45a6-9c64-0a896c6c0d72,"[Francisco Solis, Diego Solis]",6,5d342171-949f-4626-beff-983d607b60b8,"[Edgar Labra, Rodrigo Sánchez]",10
7,B,ff951871-2e1e-45a6-9c64-0a896c6c0d72,"[Francisco Solis, Diego Solis]",10,127eb4d3-a4a4-4607-bba5-1b9d7bdea6e0,"[Juan Pablo Bullet, JP Bullet Dos]",1
8,B,ff951871-2e1e-45a6-9c64-0a896c6c0d72,"[Francisco Solis, Diego Solis]",3,8f730018-9774-4486-83c5-ff5fc9d25dbb,"[Alonso Calderon, Lennin Bustos]",10
9,B,5d342171-949f-4626-beff-983d607b60b8,"[Edgar Labra, Rodrigo Sánchez]",10,127eb4d3-a4a4-4607-bba5-1b9d7bdea6e0,"[Juan Pablo Bullet, JP Bullet Dos]",2


In [28]:
df_open.shape

(48, 7)

# Paridad entre grupos. Categoria libre

In [29]:
#Make the stats for separate and anfter join them
df_team1 = df_open[['group', 'team1_id', 'team1', 'points_team1']]
df_team1 = df_team1.rename(columns={
    'team1_id': 'team_id',
    'team1': 'team',
    'points_team1': 'points_scored'
})
df_team1['points_received'] = df_open['points_team2']

df_team2 = df_open[['group', 'team2_id', 'team2', 'points_team2']]
df_team2 = df_team2.rename(columns={
    'team2_id': 'team_id',
    'team2': 'team',
    'points_team2': 'points_scored'
})
df_team2['points_received'] = df_open['points_team1']

df_paridad_open = pd.concat([df_team1, df_team2], ignore_index=True)

In [30]:
df_paridad_open

,group,team_id,team,points_scored,points_received
0,A,5083a5a9-e528-4995-a0f4-8ef47e26c2f0,"[Gerardo Martínez, Jesús Arguellas]",10,1
1,A,78062676-c1f6-4d63-bb8d-3249f2f493d1,"[José Manuel Vargas, Ernesto Oñate]",3,10
2,A,5242ad66-9a87-4135-bb39-c80570496f4a,"[Jorge Rios, JR PAREJA DOS]",2,10
3,A,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",3,10
4,A,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",10,7
...,...,...,...,...,...
91,H,ab7394cf-a02f-498b-b70f-132a57162466,"[Manuel Méndez, Eduardo Gutierrez]",0,0
92,H,7b28dec2-cda8-4bbd-b10a-7c775ea3a61a,"[Iván Zuñiga, Alejandro Flores]",5,10
93,H,9bc13149-5e99-4bef-b9a0-3fb5b5730245,"[Edgar García, Hugo Eduardo Arizmendi]",10,0
94,H,9bc13149-5e99-4bef-b9a0-3fb5b5730245,"[Edgar García, Hugo Eduardo Arizmendi]",10,2


In [31]:
df_paridad_open = df_paridad_open.groupby(['group', 'team_id', 'team']).agg({
    'points_scored': 'sum',
    'points_received': 'sum'
}).reset_index()
df_paridad_open = df_paridad_open.sort_values(['group', 'points_scored']).reset_index(drop=True)

In [32]:
df_paridad_open['points_diff'] = abs(df_paridad_open['points_scored'] - df_paridad_open['points_received'])

In [33]:
df_paridad_open.to_csv('/content/drive/MyDrive/TDLF2025/DataCharts/data_matches_open.csv', index=False, encoding='utf-8')

In [34]:
df_paridad_open

,group,team_id,team,points_scored,points_received,points_diff
0,A,5242ad66-9a87-4135-bb39-c80570496f4a,"[Jorge Rios, JR PAREJA DOS]",14,30,16
1,A,74e48e68-ee6b-4282-aaad-00c87315c036,"[Ulises Medrano, Paul Solorio]",14,27,13
2,A,78062676-c1f6-4d63-bb8d-3249f2f493d1,"[José Manuel Vargas, Ernesto Oñate]",23,18,5
3,A,5083a5a9-e528-4995-a0f4-8ef47e26c2f0,"[Gerardo Martínez, Jesús Arguellas]",30,6,24
4,B,127eb4d3-a4a4-4607-bba5-1b9d7bdea6e0,"[Juan Pablo Bullet, JP Bullet Dos]",8,30,22
5,B,ff951871-2e1e-45a6-9c64-0a896c6c0d72,"[Francisco Solis, Diego Solis]",19,21,2
6,B,5d342171-949f-4626-beff-983d607b60b8,"[Edgar Labra, Rodrigo Sánchez]",26,18,8
7,B,8f730018-9774-4486-83c5-ff5fc9d25dbb,"[Alonso Calderon, Lennin Bustos]",30,14,16
8,C,ac31ffff-3630-417c-bf43-e441c40e5f65,"[Yonatan Ángeles, Daniel Sánchez Dos]",14,30,16
9,C,13e15284-a05a-4206-9b34-c7d0fab83a3e,"[Adrián Arreola, Marcos Rojo]",23,22,1


In [35]:
# Agrupamos los datos por group_id y calculamos la desviación estándar de la diferencia entre los puntos hechos y recibidos
std_por_grupo_open = df_paridad_open.groupby('group')['points_diff'].std().round(2)

In [36]:
std_por_grupo_open.rename('parity_index', inplace=True)

,parity_index
group,
A,7.85
B,8.79
C,8.12
D,1.73
E,8.04
F,11.09
G,2.52
H,11.80


In [37]:
std_por_grupo_open = pd.DataFrame(std_por_grupo_open). reset_index()
std_por_grupo_open

,group,parity_index
0,A,7.85
1,B,8.79
2,C,8.12
3,D,1.73
4,E,8.04
5,F,11.09
6,G,2.52
7,H,11.80


In [47]:
std_por_grupo_open.to_json('/content/drive/MyDrive/TDLF2025/DataCharts/parity_open.json', orient='records')

In [38]:
# Crear la figura
fig = go.Figure()

# Agregar los datos al gráfico de barras
fig.add_trace(go.Bar(x=std_por_grupo_open['group'],
                     y=std_por_grupo_open['parity_index'],
                     marker_color='#c22322',  # Color de las barras
                     text=std_por_grupo_open['parity_index'],  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

fig.update_layout(title='¿Qué tan parejos estuvieron los grupos en la categoría Libre?',
                  xaxis_title='Grupo',
                  yaxis_title='Indice de paridad',
                  plot_bgcolor='#241111',  # Fondo negro
                  paper_bgcolor='#000000',  # Fondo del área del gráfico negro
                  font=dict(color='white'),  # Color del texto
                  width=1080,
                  height=720,
                  )

# Ajustar el espaciado entre las etiquetas de los ticks del eje x
fig.update_xaxes(dtick=1)  # Establecer dtick en 1 para mostrar todas las etiquetas

# Guardar y mostrar el gráfico
pio.show(fig)

# Extracción de parejas por grupo (Libre) puntos hechos VS puntos recibidos

In [39]:
group_a_open = extract_group(df_paridad_open, 'A')
group_b_open = extract_group(df_paridad_open, 'B')
group_c_open = extract_group(df_paridad_open, 'C')
group_d_open = extract_group(df_paridad_open, 'D')
group_e_open = extract_group(df_paridad_open, 'E')
group_f_open = extract_group(df_paridad_open, 'F')
group_g_open = extract_group(df_paridad_open, 'G')
group_h_open = extract_group(df_paridad_open, 'H')

## Puntos hechos vs Puntos recibidos (Libre)

In [40]:
fig = go.Figure()

# Agregar los datos de los puntos hechos al gráfico de barras
fig.add_trace(go.Bar(x=group_d_open['team'],
                     y=group_d_open['points_scored'],
                     name = 'Puntos hechos',
                     marker_color='#06FF74',  # Color de las barras
                     text=group_d_open['points_scored'],  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

# Agregar los datos de los puntos recibidos al gráfico de barras
fig.add_trace(go.Bar(x=group_d_open['team'],
                     y=group_d_open['points_received'],
                     name = 'Puntos recibidos',
                     marker_color='#c22322',  # Color de las barras
                     text=group_d_open['points_received'],  # Texto que se mostrará encima de cada barra
                     textposition='auto',  # Posición del texto
))

fig.update_layout(title='Puntos totales por equipo',
                  xaxis_title='Equipo',
                  yaxis_title='Puntos',
                  #xaxis_tickangle=-90,
                  plot_bgcolor='#000000',  # Fondo negro
                  paper_bgcolor='#000000',  # Fondo del área del gráfico negro
                  font=dict(color='white'),  # Color del texto
                  width=1280,
                  height=720,
                  )

# Ajustar el espaciado entre las etiquetas de los ticks del eje x
fig.update_xaxes(dtick=1)  # Establecer dtick en 1 para mostrar todas las etiquetas

# Guardar y mostrar el gráfico
pio.show(fig)

# Sinergia por pareja (Libre)

$(\frac{Ph - Pr + PJ}{Cs})*100$

Donde:

* $Ph$ = Puntos hechos por el equipo
* $Pr$ = Puntos recibidos por el equipo
* $PJ$ = Partidos Jugados por el equipo (para dar ese pequeño empujon)
* $Cs$ = Constante de sinergia que representa una sinergia perfecta (30)

**Se multipplica por 100 ya que la sinergia de un equipo se mide en porcentaje**

In [41]:
sinergia_df_open = df_paridad_open[['team','points_scored', 'points_received']].copy()
sinergia_df_open['sinergy'] = (((sinergia_df_open['points_scored'] - sinergia_df_open['points_received']+3) / 30) * 100).round(2)
sinergia_df_open['team'] = sinergia_df_open['team'].apply(lambda x: '-'.join(x.strip('[]').split(', ')))
sinergia_df_open

,team,points_scored,points_received,sinergy
0,Jorge Rios-JR PAREJA DOS,14,30,-43.33
1,Ulises Medrano-Paul Solorio,14,27,-33.33
2,José Manuel Vargas-Ernesto Oñate,23,18,26.67
3,Gerardo Martínez-Jesús Arguellas,30,6,90.00
4,Juan Pablo Bullet-JP Bullet Dos,8,30,-63.33
5,Francisco Solis-Diego Solis,19,21,3.33
6,Edgar Labra-Rodrigo Sánchez,26,18,36.67
7,Alonso Calderon-Lennin Bustos,30,14,63.33
8,Yonatan Ángeles-Daniel Sánchez Dos,14,30,-43.33
9,Adrián Arreola-Marcos Rojo,23,22,13.33


In [42]:
# Crear una figura de Plotly
fig = go.Figure()

# Agregar barras para la sinergia positiva
fig.add_trace(go.Bar(
    x=sinergia_df_open['team'],
    y=sinergia_df_open['sinergy'].apply(lambda x: max(x, 0)),  # Tomar solo los valores positivos
    name='Sinergia Positiva',
    marker_color='rgba(0, 111, 231, 0.9)',  # Color para las barras positivas
    text=sinergia_df_open['sinergy'].apply(lambda x: f'{x}%'),  # Texto que se mostrará encima de cada barra
    textposition='auto',  # Posición del texto
    textfont=dict(color='white')
))

# Agregar barras para la sinergia negativa
fig.add_trace(go.Bar(
    x=sinergia_df_open['team'],
    y=sinergia_df_open['sinergy'].apply(lambda x: min(x, 0)),  # Tomar solo los valores negativos
    name='Sinergia Negativa',
    marker_color='rgba(254, 107, 9, 0.9)',  # Color para las barras negativas
    text=sinergia_df_open['sinergy'].apply(lambda x: f'{x}%'),  # Texto que se mostrará encima de cada barra
    textposition='auto',  # Posición del texto
    textfont=dict(color='white')
))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Sinergia de Equipos',
    xaxis=dict(title='Equipo'),
    yaxis=dict(title='Sinergia'),
    xaxis_tickangle=-90,
    plot_bgcolor='#000000',  # Fondo negro
    paper_bgcolor='#000000',  # Fondo del área del gráfico negro
    font=dict(color='white'),  # Color del texto
    width=1280,
    height=720
)

fig.update_xaxes(dtick=1)

# Mostrar el gráfico
fig.show()

# Explicación de la sinergia

**¡Entiende el % de Sinergia de tu Equipo en el Torneo! 🎾🌟**

La sinergia es un indicador que mide cómo de bien trabaja tu equipo en la cancha. Se calcula comparando los puntos que hacen (Ph) con los que reciben (Pr), y se expresa en porcentaje. Aquí te explicamos qué significa cada rango:

1. 🔥 Sinergia Positiva Alta (40% o más)
Tu equipo está en la zona de excelencia.

Ejemplo: Si tu equipo tiene 60% de sinergia, significa que por cada 10 puntos jugados, ganan 6 más de los que pierden.

**¿Qué pasa aquí?**

Dominan los partidos con estrategias claras y comunicación fluida.

Suelen cerrar los juegos con ventajas amplias.

**Recomendación:** ¡Sigan así! Analicen sus táctias estrella para mantener el ritmo en fases decisivas.

2. ⚖️ Sinergia Neutral (entre -10% y +10%)
Tu equipo está en equilibrio... pero hay margen para crecer.

Ejemplo: Una sinergia de 0% indica que, por cada 10 puntos, ganan y pierden lo mismo.

**¿Qué pasa aquí?**

Los partidos son reñidos: a veces ganan por poco, a veces pierden por detalles.

Falta un "extra" para convertirse en un equipo dominante.

**Recomendación:** Enfóquense en mejorar momentos clave (ej: saques decisivos o remates bajo presión).

3. 🆘 Sinergia Negativa Grave (-50% o menos)
Es momento de ajustes, pero no de rendirse.

Ejemplo: Con -60% de sinergia, por cada 10 puntos, el equipo pierde 6 más de los que gana.

**¿Qué pasa aquí?**

Errores repetidos, falta de coordinación o estrategias que no funcionan.

¡Ojo! Esto no significa que el equipo sea "malo": puede ser falta de práctica o química temporal.

**Recomendación:** Revisen roles, practiquen situaciones específicas y busquen asesoría técnica.

¿Y si mi equipo está entre -50% y -10%?

Es una señal de alerta, pero no catastrófica. Por ejemplo:

-30% de sinergia significa que, por cada 10 puntos, pierden 3 más de los que ganan.

**Acción clave:** Identifiquen patrones (ej: fallos en defensa o falta de definición en ataques).

**Recordatorio Final 💡**
El % de sinergia no define para siempre a tu equipo: Es una foto del momento. ¡Con trabajo, pueden subir!

Usen este dato para mejorar, no para desmotivarse. Hasta los mejores equipos tienen días malos.

¡A jugar se aprende jugando! 🏆✨

# ==========COMPARACIONES ENTRE CATEGORIAS==========

#¿Qué categoría tiene mejor sinergia?

In [43]:
sinergia_mean_seniors = sinergia_df_seniors['sinergy'].mean()
sinergia_mean_open = sinergia_df_open['sinergy'].mean()
sinergia_groups = {
    'sinergy_open': float(sinergia_mean_open),
    'sinegy_seniors': float(sinergia_mean_seniors)
}
sinergia_groups['sinergy_open'] = round(sinergia_groups['sinergy_open'], 4)
sinergia_groups['sinegy_seniors'] = round(sinergia_groups['sinegy_seniors'], 4)

sinergia_groups

{'sinergy_open': 10.0006, 'sinegy_seniors': 9.9994}

In [44]:
figg = go.Figure([go.Bar(x=['Libre','50 y más'], y=[sinergia_groups['sinergy_open'], sinergia_groups['sinegy_seniors']], text=[sinergia_groups['sinergy_open'], sinergia_groups['sinegy_seniors']], textposition='auto')])

# Personalizar el gráfico
figg.update_layout(
    title='Sinergia en ambas categorias',
    xaxis_title='Categorías',
    yaxis_title='Sinergia',
    bargap=0.4  # Ajustar el espacio entre las barras
)

# Personalizar el diseño del gráfico
figg.update_layout(
    title='Sinergia de Equipos',
    xaxis=dict(title='Grupo'),
    yaxis=dict(title='Sinergia'),
    plot_bgcolor='#000000',  # Fondo negro
    paper_bgcolor='#000000',  # Fondo del área del gráfico negro
    font=dict(color='white'),  # Color del texto
    width=1280,
    height=720
)

figg.update_xaxes(dtick=1)

# Mostrar el gráfico
figg.show()

# ¿Qué categoría hizo más puntos?

In [45]:
points_seniors = df_paridad_seniors['points_scored'].sum()
points_open = df_paridad_open['points_scored'].sum()
points_groups = {
    'points_open': int(points_open),
    'points_seniors': int(points_seniors)
}
points_groups

{'points_open': 616, 'points_seniors': 309}

In [46]:
figg = go.Figure([go.Bar(x=['Libre','50 y más'], y=[points_groups['points_open'], points_groups['points_seniors']], text=[points_groups['points_open'], points_groups['points_seniors']], textposition='auto')])

# Personalizar el gráfico
figg.update_layout(
    title='Sinergia en ambas categorias',
    xaxis_title='Categorías',
    yaxis_title='Sinergia',
    bargap=0.4  # Ajustar el espacio entre las barras
)

# Personalizar el diseño del gráfico
figg.update_layout(
    title='Sinergia de Equipos',
    xaxis=dict(title='Grupo'),
    yaxis=dict(title='Sinergia'),
    plot_bgcolor='#000000',  # Fondo negro
    paper_bgcolor='#000000',  # Fondo del área del gráfico negro
    font=dict(color='white'),  # Color del texto
    width=1280,
    height=720
)

figg.update_xaxes(dtick=1)

# Mostrar el gráfico
figg.show()